In [1]:
import openai
import os

# from dotenv import load_dotenv, find_dotenv
# _ = load_dotenv(find_dotenv()) # read local .env file

# openai.api_key  = os.getenv('OPENAI_API_KEY')
openai.api_key  = ""

In [39]:
client = openai.AsyncOpenAI(
  api_key=openai.api_key,  # this is also the default, it can be omitted
)


async def get_completion(prompt_, model_="gpt-3.5-turbo"):
    messages_ = [{"role": "user", "content": prompt_}]
    response_ = await client.chat.completions.create(
        model=model_, 
        messages=messages_,
        temperature=0.00000001,
    )
    return response_.choices[0].message.content

# Experiments

In [20]:
str_term = "(λx.((λy.((λz.z) x)) (λa.a)))"
next_step_term = "(λx.((λy.y) (λa.a)))"

prompt = f"""
Please generate the next step of reduction a lambda term. Provide only term expression. Use the leftmost outermost strategy.

Lambda term: '''{str_term}'''
"""
response = await get_completion(prompt)
print(f"expected output: {next_step_term}")
print(f"model output: {response}")

expected output: (λx.((λy.y) (λa.a)))
model output: (λx.((λy.y) (λa.a)))


In [23]:
str_term = "(λx.((λy.((λz.z) x)) (λa.a)))"
next_step_term = "(λx.((λy.y) x))"

prompt = f"""
Please generate the next step of reduction a lambda term. Provide only term expression. Use the rightmost innermost (RI) strategy.
The RI strategy use the latest redex for reduction. Pay attention on this fact.

Lambda term: '''{str_term}'''
"""
response = await get_completion(prompt)
print(f"expected output: {next_step_term}")
print(f"model output: {response}")

expected output: (λx.((λy.y) x))
model output: (λx.((λy.y) (λa.a)))


In [25]:
str_term = "(λx.((λy.[(λz.z) x]) (λa.a)))"
prompt = f"""
Please generate the next step of reduction a lambda term.
The reduction redex is selected with square brackets.

Provide only term expression.

Lambda term: '''{str_term}'''
"""
response = await get_completion(prompt)
print(f"model output: {response}")

model output: (λx.((λy.x) (λa.a)))


In [45]:
str_term = "[λx.((λy.((λz.z) x)) (λa.a))]"
prompt = f"""
Please generate the next step of reduction a lambda term.
The reduction redex is selected with square brackets.

Provide only term expression.

Lambda term: '''{str_term}'''
"""
response = await get_completion(prompt)
print(f"model output: {response}")

model output: (λx.((λy.((λz.z) x)) (λa.a))) [x/(λa.a)]


In [47]:
str_term = "[λx.((λy.((λz.z) x)) (λa.a))]"
prompt = f"""
Please execute a lambda term, do one step of execution using a selected redex.
The redex is selected with square brackets.
Provide only the next term expression.

Lambda term: '''{str_term}'''
"""
response = await get_completion(prompt)
print(f"model output: {response}")

model output: (λx.((λy.((λz.z) x)) (λa.a))) 
→ (λx.((λz.z) x) (λa.a))


In [48]:
import tiktoken


str_term = "(λx.((λy.((λz.z) x)) (λa.a)))"
enc_tiktoken = tiktoken.encoding_for_model("gpt-3.5-turbo")
term_enc = enc_tiktoken.encode(str_term)

In [52]:
for enc_token in term_enc:
    print(f"{enc_token}: {enc_tiktoken.decode([enc_token,])}")

7: (
34586: λ
87: x
13: .
1209: ((
34586: λ
88: y
13: .
1209: ((
34586: λ
89: z
4025: .z
8: )
865:  x
595: ))
320:  (
34586: λ
64: a
5973: .a
7861: )))


In [53]:
str_term = "( λ x . ( ( λ y . ( ( λ z . z ) x ) ) ( λ a . a ) ) )"
enc_tiktoken = tiktoken.encoding_for_model("gpt-3.5-turbo")
term_enc = enc_tiktoken.encode(str_term)
for enc_token in term_enc:
    print(f"{enc_token}: {enc_tiktoken.decode([enc_token, ])}")

7: (
49438:  λ
865:  x
662:  .
320:  (
320:  (
49438:  λ
379:  y
662:  .
320:  (
320:  (
49438:  λ
1167:  z
662:  .
1167:  z
883:  )
865:  x
883:  )
883:  )
320:  (
49438:  λ
264:  a
662:  .
264:  a
883:  )
883:  )
883:  )


In [61]:
str_term = "[ λ x . ( ( λ y . ( ( λ z . z ) x) ) ( λ a . a ) ) ]"
prompt = f"""
Please execute a lambda term, do one step of execution using a selected redex.
The redex is selected with square brackets.
Provide only the next term expression.

Lambda term: '''{str_term}'''
"""
response = await get_completion(prompt)
print(f"model output: {response}")

model output: ( λ y . ( ( λ z . z ) ( λ a . a ) ) )


In [62]:
str_term = "( λ x . ( ( λ y . ( ( λ z . z ) x) ) ( λ a . a ) ) )"
prompt = f"""
Please execute a lambda term, do one step of execution using a selected redex.
Provide only the next term expression.

Lambda term: '''{str_term}'''
"""
response = await get_completion(prompt)
print(f"model output: {response}")

model output: ( ( λ y . ( ( λ z . z ) x) ) ( λ a . a ) )
